## Statistics for Value Mappings

In [1]:
import pandas as pd
from gdc_utils import get_gdc_data
from value_matcher import TFIDFMatcher, EmbeddingMatcher, EditMatcher, LLMMatcher
from value_matcher_interface import MatcherInterface

/Users/rlopez/opt/anaconda3/envs/askem/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
column_mapping = {
    #"Proteomics_Participant_ID": "case_submitter_id",
    #"Age": "age_at_diagnosis",
    "Gender": "gender",
    "Race": "race",
    "Ethnicity": "ethnicity",
    #"(none)": "(none)",
    "Histologic_Grade_FIGO": "tumor_grade",
    "tumor_Stage-Pathological": "ajcc_pathologic_stage",
    "Path_Stage_Reg_Lymph_Nodes-pN": "ajcc_pathologic_n",
    "Path_Stage_Primary_Tumor-pT": "ajcc_pathologic_t",
    "Tumor_Focality": "tumor_focality",
    #"Tumor_Size_cm": "tumor_largest_dimension_diameter",
    "Tumor_Site": "tissue_or_organ_of_origin",
    "Histologic_type": "morphology",
    #"Case_excluded": "(none)"
}

gdc_data = get_gdc_data(column_mapping.values())
dataset = pd.read_csv('../data/use_case1/dou.csv')
dataset = dataset[column_mapping.keys()]

In [3]:
import json

GDC_SCHEMA_PATH = '../data/use_case1/gdc_schema.json'


def read_gdc_schema():
    with open(GDC_SCHEMA_PATH) as json_file:
        gdc_schema = json.load(json_file)

    return gdc_schema


def get_gdc_data(column_names):
    gdc_schema = read_gdc_schema()
    gdc_data = {}

    for column_name in column_names:
        gdc_values = get_gdc_values(column_name, gdc_schema)
        gdc_data[column_name] = gdc_values

    return gdc_data


def get_gdc_values(column_name, gdc_schema):
    for key, values in gdc_schema.items():
        for key in values['properties'].keys():
            if column_name == key:
                value_metadata = values['properties'][column_name]
                if "enumDef" in value_metadata:
                    return value_metadata['enumDef']
                elif "type" in value_metadata and value_metadata['type'] == 'number':
                    return None

    return None

In [5]:
data = get_gdc_data(['morphology'])

In [9]:
data['morphology']['8380/3']['description']

'An epithelial neoplasm that arises from the ovary characterized by the presence of glandular or cystic spaces which contain atypical glandular epithelial cells resembling endometrial cells. The surrounding ovarian stroma is often fibrotic. There is no evidence of stromal invasion.'